# RAG Strategy Comparison

Compare **Plain RAG**, **Graph RAG**, and **No-RAG** on the same queries. Use this notebook for systematic evaluation; for single-query exploration use `AgenticExplainability_Usage.ipynb`.

## 1. Setup

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import json
from pathlib import Path

def find_project_root():
    for start in [Path.cwd()] + list(Path.cwd().parents):
        if (start / "agentic_explain").is_dir() and (start / "use_case").is_dir():
            return start
    return Path.cwd()

PROJECT_ROOT = find_project_root()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

from config.load_secrets import load_secrets, get_gurobi_env_kwargs
from use_case.staffing_model import STAFFING_DATA_DIR, STAFFING_OUTPUTS_DIR, load_raw_data, process_data, build_gurobi_model

load_secrets()
DATA_DIR = STAFFING_DATA_DIR
OUTPUTS_DIR = STAFFING_OUTPUTS_DIR
OUTPUTS_DIR.mkdir(parents=True, exist_ok=True)

raw = load_raw_data(DATA_DIR)
inputs = process_data(**raw)
env_kwargs = get_gurobi_env_kwargs()

with open(OUTPUTS_DIR / "baseline_result.json", "r", encoding="utf-8") as f:
    baseline_result = json.load(f)

openai_client = __import__("openai").OpenAI()

## 2. Build all three RAG strategies

In [ ]:
from agentic_explain.rag import build_plain_rag, build_no_rag, build_graph_rag
from agentic_explain.workflow.graph import create_workflow, invoke_workflow
from use_case.staffing_model import StaffingGraphRAGConfig

REBUILD_PLAIN = False
REBUILD_GRAPH = False

py_path = PROJECT_ROOT / "use_case" / "staffing_model" / "staffing_model.py"
lp_path = OUTPUTS_DIR / "model.lp"
mps_path = OUTPUTS_DIR / "model.mps"
rag_dir = OUTPUTS_DIR / "rag_index"
graph_dir = OUTPUTS_DIR / "graph_rag_index"

plain_strategy = build_plain_rag(py_path, lp_path=lp_path, mps_path=mps_path, data_dir=DATA_DIR, persist_dir=rag_dir, force_rebuild=REBUILD_PLAIN)
no_rag_strategy = build_no_rag(data_dir=DATA_DIR)
graph_config = StaffingGraphRAGConfig(DATA_DIR)
graph_strategy = build_graph_rag(graph_config, lp_path=lp_path, persist_dir=graph_dir, force_rebuild=REBUILD_GRAPH)

strategies = {"plain_rag": plain_strategy, "no_rag": no_rag_strategy, "graph_rag": graph_strategy}
print("Strategies built:", list(strategies.keys()))

## 3. Single-query comparison

In [ ]:
query = "Why was Josh not staffed on Ipp IO Pilot in week 6?"

results = {}
for name, strategy in strategies.items():
    workflow = create_workflow(openai_client=openai_client, rag_strategy=strategy, baseline_result=baseline_result,
        data_dir=str(DATA_DIR), build_model_fn=build_gurobi_model, inputs=inputs, env_kwargs=env_kwargs, outputs_dir=str(OUTPUTS_DIR))
    state = invoke_workflow(workflow, query, baseline_result=baseline_result)
    results[name] = state
    print(f"{name}: status={state.get('counterfactual_status')}")

print("\nFinal summaries:")
for name, state in results.items():
    print(f"\n--- {name} ---")
    print(state.get("final_summary", "(none)")[:500])

## 4. Batch evaluation (optional)

In [ ]:
from agentic_explain.evaluation.helpers import load_eval_queries, run_batch_evaluation

queries_path = PROJECT_ROOT / "agentic_explain" / "evaluation" / "queries.json"
eval_queries = load_eval_queries(queries_path)
print(f"Loaded {len(eval_queries)} queries")

# Run batch for one strategy (uncomment and run):
# workflow_plain = create_workflow(openai_client=openai_client, rag_strategy=plain_strategy, baseline_result=baseline_result,
#     data_dir=str(DATA_DIR), build_model_fn=build_gurobi_model, inputs=inputs, env_kwargs=env_kwargs, outputs_dir=str(OUTPUTS_DIR))
# batch_results = run_batch_evaluation(workflow_plain, eval_queries[:5], baseline_result, OUTPUTS_DIR / "eval_plain.json", rag_strategy_name="plain_rag")